In [12]:
#Lane finding program

In [13]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [14]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [15]:
# Performs simple linear regression on a set of lines.
def lines_linreg(lines_array):
    x = np.reshape(lines_array[:, [0, 2]], (1, len(lines_array) * 2))[0]
    y = np.reshape(lines_array[:, [1, 3]], (1, len(lines_array) * 2))[0]
    #print ("this is  the shape of x: ",x.shape)
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y)[0]
    x = np.array(x)
    y = np.array(x * m + c)
    return x, y, m, c

In [16]:
# Takes an array of lines and gives back lines which have a valid slope, given a range.
def slope_filter(lines_array, positive, min_slope, max_slope):
    '''
    np.apply_along_axis is a function which is use to apply the supplied function to the axis specified
    here we supply a function (lambda slop: (slope[3] - slope[1]) / (slope[2] - slope[0]) which is going to calculate the 
    slope of a line at a time and store it in slopes 
    
    if positive is true :
    it calculates the right side lane slope
    if positive is false :
    it calculates the leftside lane slope
    
    after calculating the slopes . it return the value of slope which 
    '''
    slopes = np.apply_along_axis(lambda slope: (slope[3] - slope[1]) / (slope[2] - slope[0]), 2, lines_array)

    
    if positive:
        slopes[slopes > max_slope] = 0
        slopes[slopes < min_slope] = 0
        lines_array =np.array(lines_array[np.where(slopes > 0)])
    else:
        slopes[slopes < -max_slope] = 0
        slopes[slopes > -min_slope] = 0
        lines_array = np.array(lines_array[np.where(slopes < 0)])

    #print ("the shape of the slope",lines_array.shape)
    return lines_array

In [17]:
'''
#helper functions
import math

def grayscale(img):
    
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    
def canny(img, low_threshold, high_threshold): # use this function to find the edges in the input image frame
    
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):# to smooth and blur the image to remove noices and get fine lines 
    
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):# to bitwise add the region of interest to the input stream of data
    
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

'''
#helper functions
import math

def grayscale(img):# use this function to convert the BGR image to grayscale  image
    
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



In [18]:
'''


this is a replacement function of the provided weighted_img fn
steps of work
    1.it takes in the raw image on which you like to draw the points
    2.top and bottom points of left and right are feed into the function
    3.it creats a np.zero array of the same size as image provided
    4.draw line on the blank image 
    5.do cv2.addWeighted function which adds both the image and the blank image with hough lines drawn on it along with 
    α, β, λ 


'''
def drawWeightedImage(image,bottom_left_point,top_left_point,bottom_right_point,top_right_point):
    
    
    toDraw = np.zeros_like(image)
    
    cv2.line(toDraw,(bottom_left_point[0],bottom_left_point[1]),(top_left_point[0],top_left_point[1]),[255,0,0],10)
    cv2.line(toDraw,(bottom_right_point[0],bottom_right_point[1]),(top_right_point[0],top_right_point[1]),[255,0,0],10)
        
    return cv2.addWeighted(image, 0.8, toDraw, 1., 0.)



In [19]:
'''

the use of this function is to calcuate the bottom and top 
points of the right and left lines ,so that we can draw 
lines using this points
 
steps of work:
    1.it finds the top most and bottom most y point by min and max fn
    2.create a np.array of x and y values using the formula
        y = mx+c
        x = (y-c)/m
    returns the final values
    
'''
def findPointsToDraw(right_y,right_c,right_m,left_y,left_c,left_m):
    
    #print ("size of \n","right_y \t",right_y.shape,"right_c \t",right_c,"right_m \t",right_m)
    min_y = np.min(right_y)# minimum y point will be the top most point
    '''
    the formula for line is y = mx +c
    
    so 
    x = (y-c)/m
    
    this is used to calculate the points of top and bottom and fromed into an np.array
    '''
    
    
    
    #                           (  y    -     c  ) /     m
    top_right_point = np.array([(min_y - right_c) / right_m, min_y], dtype=int) 

    max_y = np.max(right_y)
    bottom_right_point = np.array([(max_y - right_c) / right_m, max_y], dtype=int)

    #cv2.line(image, (bottom_right_point[0], bottom_right_point[1]), (top_right_point[0], top_right_point[1]), [255, 0, 0], 10)

    min_y = np.min(left_y)
    top_left_point = np.array([(min_y - left_c) / left_m, min_y], dtype=int)

    max_y = np.max(left_y)
    bottom_left_point = np.array([(max_y - left_c) / left_m, max_y], dtype=int)
    
    return top_right_point,bottom_right_point,top_left_point,bottom_left_point
    

In [20]:

def process_image1(image):
    
    
    # convert the input image to grayscale and apply gaussian_blur to it
    imgGray = grayscale(image)
    kernel_size = 5
    imgGray = gaussian_blur(imgGray, kernel_size)
    
    
    # apply canny edge detection to the image and dialte the edgedetected image
    lowerThreshold=80
    higherThreshold=160
    edgeImage = canny (imgGray,lowerThreshold,higherThreshold)
    edgeImage= cv2.dilate(edgeImage, cv2.getStructuringElement(cv2.MORPH_DILATE, (5, 5)))
    
    # define region of interest for both right and left side
    ratio = 5/8
    height = image.shape[0]
    width = image.shape[1]
    #print (height)
    
    roi_left = np.array([[
        (100, height),
        ((1 - ratio) * width, ratio * height),
        (.5 * width, ratio * height),
        (.5 * width, height)
    ]], dtype=np.int32)

    roi_right = np.array([[
        (.5 * width, height),
        (.5 * width, ratio * height),
        (ratio * width, ratio * height),
        (width, height)
    ]], dtype=np.int32)
    
    
    # create a mask area
    roiImageRight = region_of_interest(edgeImage, roi_right)
    
    
    # create a mask area
    roiImageLeft = region_of_interest(edgeImage, roi_left)
    

    
    
    # apply hough transform to the image wiht region of interest  masked
    rho = 80
    theta = 2*np.pi/180
    threshold = 60*2*2
    min_line_len = 1
    max_line_gap = 10
    try:
        linesRight = cv2.HoughLinesP(roiImageRight, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
        
        # the linesRight is of shape (:,1,4) which has x1,y1,x2,y2 
        
        lines_right = slope_filter(linesRight, True, .5, .9) # used to filter the points which lies between the range 0.5-0.9
        right_x, right_y, right_m, right_c = lines_linreg(lines_right) # find the x , y values with respected to the desired m ,and c values
        
        
    except :
        pass
    
    # do the same thing as above but to the left side image
    try:
        linesLeft = cv2.HoughLinesP(roiImageLeft, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
        lines_left = slope_filter(linesLeft, False, 0.1, 0.9)
        left_x, left_y, left_m, left_c = lines_linreg(lines_left)
    except :
        pass
    # find the top,bottom most point to draw lines
    try:
        top_right_point,bottom_right_point,top_left_point,bottom_left_point = findPointsToDraw(right_y,
                                                                                               right_c,
                                                                                               right_m,
                                                                                               left_y,
                                                                                               left_c,
                                                                                               left_m)
        
        # draw ines on the input image and store it to a numpy array called finalImage
        finalImage = drawWeightedImage(image,bottom_left_point,top_left_point,bottom_right_point,top_right_point)
    
    
    
    except :# if there is a problem occurs , just return the input image so that our progrma continues to execute
        #pass
        return image
    

    return finalImage


In [21]:
white_output = 'FinalOutput.mp4'
nameOfTheFile = "solidWhiteRight.mp4"
clip1 = VideoFileClip(nameOfTheFile)
white_clip = clip1.fl_image(process_image1) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video FinalOutput.mp4
[MoviePy] Writing video FinalOutput.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:14<00:00, 15.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: FinalOutput.mp4 

Wall time: 16.2 s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("FinalOutput.mp4"))